In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload
import matplotlib.pyplot as plt

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import model_params as params
import utils.resnet_orig as resnet
import utils.gmm_helpers as gmm_helpers
#import utils.gmm as GMM

from tensorboardX import SummaryWriter
%load_ext line_profiler

In [50]:
device = torch.device('cuda:6')
gmm = torch.load('SavedModels/GMM/gmm_MNIST_n1000_data_used60000_augm_flagTrue_alg_scikit.pth').to(device)
#gmm = models.GMM(500, 784).to(device)

In [3]:
train_loader = dl.MNIST(train=True, batch_size=512, augm_flag=True)
test_loader = dl.MNIST(train=False, augm_flag=False)

In [4]:
lr = 0.001

param_groups = [{'params':gmm.mu,'lr':lr, 'weight_decay':0.},
                {'params':gmm.logvar,'lr':lr, 'weight_decay':0.}]
optimizer = optim.SGD(param_groups,  momentum=.9)

In [5]:
_, (data, _) = enumerate(train_loader).__next__()
gmm.mu.data = data[:gmm.K].view(-1, 784).to(device)

In [59]:
#criterion = nn.NLLLoss()
gmm.train()

train_loss = 0
correct = 0

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()
    output = torch.logsumexp( gmm(data.view(data.shape[0], -1)), 0 ) - 625.7730167
    loss_CEDA = torch.logsumexp(gmm(torch.rand(100, 784, device=device)), 0).mean() - 625.7730167
    #loss_CEDA = 0.
    loss = -output.mean() + loss_CEDA
    loss.backward()
    #optimizer.step()

    train_loss += loss.item()


In [52]:
print(gmm.logvar.mean())
print(gmm.mu.mean())
print(train_loss)
print(output.mean())
print(gmm.mu.grad.abs().max())
print(gmm.logvar.grad.abs().max(0))

tensor(-3.4188, device='cuda:6', grad_fn=<MeanBackward1>)
tensor(0.1372, device='cuda:6', grad_fn=<MeanBackward1>)
-151910.87255859375
tensor(269.5195, device='cuda:6', grad_fn=<MeanBackward1>)


AttributeError: 'NoneType' object has no attribute 'abs'

In [58]:
torch.logsumexp(gmm(torch.rand(100, 784, device=device)), 0).max()

tensor(-218.7498, device='cuda:6', grad_fn=<MaxBackward1>)

Parameter containing:
tensor([-3.3965, -4.8272, -3.4398, -3.2954, -3.6526, -3.5059, -3.7981, -3.3597,
        -3.5565, -3.3831, -3.4182, -3.5783, -3.6426, -4.8337, -4.0327, -3.2136,
        -4.4995, -4.1311, -3.6154, -3.0753, -3.1343, -3.3480, -3.1363, -3.1603,
        -3.0621, -3.2979, -3.1388, -3.1792, -3.2777, -3.5853, -3.5764, -3.7955,
        -4.8774, -4.6345, -3.3428, -3.6290, -2.9044, -3.2397, -4.7661, -3.3996,
        -3.0726, -3.1795, -4.6050, -3.6613, -3.2569, -3.5025, -3.0670, -3.1325,
        -3.1865, -3.4457, -4.6319, -3.5729, -3.4273, -4.2115, -3.4319, -3.0591,
        -3.5084, -3.4843, -2.9689, -3.3666, -3.2925, -3.2154, -3.5557, -3.6253,
        -3.4599, -3.7843, -3.5400, -3.3434, -3.3730, -3.4570, -2.9498, -3.3055,
        -3.5661, -3.2897, -3.4184, -4.7851, -3.2862, -3.3159, -3.1958, -3.2200,
        -3.6267, -3.0640, -3.2543, -3.1370, -3.0156, -3.3928, -3.2684, -3.4587,
        -3.3792, -3.5496, -3.0836, -3.3368, -3.1415, -3.3738, -3.3695, -3.6379,
        -3.3022, -

In [54]:
gmm.logvar.exp().sqrt()

tensor([0.1830, 0.0895, 0.1791, 0.1925, 0.1610, 0.1733, 0.1497, 0.1864, 0.1689,
        0.1842, 0.1810, 0.1671, 0.1618, 0.0892, 0.1331, 0.2005, 0.1054, 0.1267,
        0.1640, 0.2149, 0.2086, 0.1875, 0.2084, 0.2059, 0.2163, 0.1923, 0.2082,
        0.2040, 0.1942, 0.1665, 0.1673, 0.1499, 0.0873, 0.0985, 0.1880, 0.1629,
        0.2341, 0.1979, 0.0923, 0.1827, 0.2152, 0.2040, 0.1000, 0.1603, 0.1962,
        0.1736, 0.2158, 0.2088, 0.2033, 0.1786, 0.0987, 0.1676, 0.1802, 0.1218,
        0.1798, 0.2166, 0.1730, 0.1751, 0.2266, 0.1858, 0.1928, 0.2003, 0.1690,
        0.1632, 0.1773, 0.1507, 0.1703, 0.1879, 0.1852, 0.1775, 0.2288, 0.1915,
        0.1681, 0.1930, 0.1810, 0.0914, 0.1934, 0.1905, 0.2023, 0.1999, 0.1631,
        0.2161, 0.1965, 0.2084, 0.2214, 0.1833, 0.1951, 0.1774, 0.1846, 0.1695,
        0.2140, 0.1885, 0.2079, 0.1851, 0.1855, 0.1622, 0.1918, 0.1932, 0.1751,
        0.1559, 0.1987, 0.1746, 0.1561, 0.1774, 0.2237, 0.2266, 0.1542, 0.1634,
        0.1842, 0.2276, 0.1631, 0.2072, 

In [ ]:
def test(model, device, test_loader, min_conf=.1):
    model.eval()
    test_loss = 0
    correct = 0.
    av_conf = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            c, pred = output.max(1, keepdim=True) # get the index of the max log-probability
            correct += (pred.eq(target.view_as(pred))*(c.exp()>min_conf)).sum().item()
            av_conf += c.exp().sum().item()
            
    test_loss /= len(test_loader.dataset)
    av_conf /= len(test_loader.dataset)
    correct /= len(test_loader.dataset)
    
    return correct, av_conf, test_loss

In [20]:
output.mean()

tensor(924.5433, device='cuda:6', grad_fn=<MeanBackward1>)

In [102]:
reload(models)

<module 'utils.models' from '/home/alexm/project/notebooks/gmm-robust/utils/models.py'>